In [1]:
import numpy as np
import pandas as pd

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def loadSimpleData():
    dataMat = np.matrix([[1.,2.1],[2.,1.1],[1.3,1.],[1.,1.],[2.,1.]])
    classLabels = [1.,1.,-1.,-1.,1.]
    return dataMat,classLabels

In [3]:
dataMat,classLabels = loadSimpleData()

In [78]:
# 根据不同特征(dimen)和阈值,以及大于小于(gt,lt)进行分类
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    retArray = np.ones((np.shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1
    return retArray

def buildStump(dataArr,classLabels,D):
    dataMatrix = np.mat(dataArr);labelMat = np.mat(classLabels).T
    m,n = dataMatrix.shape
    numSteps = 10.0;bestStump={};bestClassEst = np.mat(np.zeros((m,1)))
    minError = np.inf
    for i in range(n):
        rangeMin = dataMatrix[:,i].min();rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax - rangeMin) / numSteps 
        for j in range(-1,int(numSteps + 1)):
            for inequal in ['gt','lt']:
                threshVal = rangeMin + (float(j) * stepSize)
                predictionVals = \
                                stumpClassify(dataMatrix,i,threshVal,inequal)
                errArr = np.mat(np.ones((m,1)))
                errArr[predictionVals == labelMat] = 0
                #print("D.T:",D.T,"errArr:",errArr)
                weightedError = float(D.T * errArr)
                #rint("split:dim %d, thresh %.2f,thresh ineqal: %s,the weighted error is %.3f"%(i,threshVal,inequal,weightedError))
                if weightedError < minError:
                    minError = weightedError
                    bestClassEst = predictionVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClassEst

In [19]:
buildStump(dataMat,classLabels,np.ones((len(dataMat),1)) /float(len(dataMat)))

split:dim 0, thresh 0.90,thresh ineqal: gt,the weighted error is 0.600
split:dim 0, thresh 0.90,thresh ineqal: lt,the weighted error is 0.400
split:dim 0, thresh 1.00,thresh ineqal: gt,the weighted error is 0.600
split:dim 0, thresh 1.00,thresh ineqal: lt,the weighted error is 0.400
split:dim 0, thresh 1.10,thresh ineqal: gt,the weighted error is 0.600
split:dim 0, thresh 1.10,thresh ineqal: lt,the weighted error is 0.400
split:dim 0, thresh 1.20,thresh ineqal: gt,the weighted error is 0.600
split:dim 0, thresh 1.20,thresh ineqal: lt,the weighted error is 0.400
split:dim 0, thresh 1.30,thresh ineqal: gt,the weighted error is 0.800
split:dim 0, thresh 1.30,thresh ineqal: lt,the weighted error is 0.200
split:dim 0, thresh 1.40,thresh ineqal: gt,the weighted error is 0.800
split:dim 0, thresh 1.40,thresh ineqal: lt,the weighted error is 0.200
split:dim 0, thresh 1.50,thresh ineqal: gt,the weighted error is 0.800
split:dim 0, thresh 1.50,thresh ineqal: lt,the weighted error is 0.200
split:

({'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, matrix([[ 0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

In [110]:
def adaBoostTrainDS(dataArr,classLabels,numIter=40):
    weakClassArr = []
    m = np.shape(dataArr)[0]
    D = np.mat(np.ones((m,1)) / m)
    aggClassEst = np.mat(np.zeros((m,1)))
    for i in range(numIter):
        bestStump,error,classEst = buildStump(dataArr,classLabels,D)
        print("D:",D.T)
        weakClassArr.append(bestStump)
        alpha = float(0.5 * np.log((1 - error) / max(error,1e-16)))#计算alpha
        bestStump['alpha'] = alpha
        #print("alpha:",alpha)
        print("classEst:",classEst)
        #print("classLabel:",classLabels)
        #print("aggClassEst:",aggClassEst)
        espon = np.exp(np.multiply(-alpha *np.mat(classLabels).T,classEst))
        print("espon",espon)
        D = np.multiply(D,espon)
        D =  D/ D.sum()
        #print("D:",D)
        #print("a * classEst",alpha * classEst)
        aggClassEst += alpha* classEst
        print ("aggClassEst: ",aggClassEst.T)
        aggErrors = np.multiply(np.sign(aggClassEst) !=  np.mat(classLabels).T,np.ones((m,1)))
        errorRate = aggErrors.sum() / m
        print ("total error:",errorRate)
        if errorRate == 0.0:break
    return weakClassArr
        

In [111]:
classifierArray = adaBoostTrainDS(dataMat,classLabels,9)

D: [[ 0.2  0.2  0.2  0.2  0.2]]
classEst: [[-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]]
espon [[ 2. ]
 [ 0.5]
 [ 0.5]
 [ 0.5]
 [ 0.5]]
aggClassEst:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error: 0.2
D: [[ 0.5    0.125  0.125  0.125  0.125]]
classEst: [[ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]]
espon [[ 0.37796447]
 [ 0.37796447]
 [ 0.37796447]
 [ 0.37796447]
 [ 2.64575131]]
aggClassEst:  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error: 0.2
D: [[ 0.28571429  0.07142857  0.07142857  0.07142857  0.5       ]]
classEst: [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]
espon [[ 0.40824829]
 [ 0.40824829]
 [ 2.44948974]
 [ 2.44948974]
 [ 0.40824829]]
aggClassEst:  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error: 0.0


In [112]:
print (classifierArray)

[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599453}, {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276565}, {'dim': 0, 'thresh': 0.90000000000000002, 'ineq': 'lt', 'alpha': 0.8958797346140273}]


In [115]:
def addClassify(dataToClass,classifierArray):
    dataMat = np.mat(dataToClass)
    m = np.shape(dataMat)[0]
    aggClassEst = np.mat(np.zeros((m,1)))
    for i in range(len(classifierArray)):
        classEst = stumpClassify(dataMat,classifierArray[i]['dim'],classifierArray[i]['thresh'],classifierArray[i]['ineq'])
        aggClassEst += classifierArray[i]['alpha'] * classEst
        print (aggClassEst)
    return np.sign(aggClassEst)
        

In [117]:
addClassify([5,5],classifierArray)

[[ 0.69314718]]
[[ 1.66610226]]
[[ 2.56198199]]


matrix([[ 1.]])